In [1]:
import os
import cv2
import scipy
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

In [2]:
base_dir = r'/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/healthy'

dst_dir = r'/mnt/labnas/DataCollection/Project/RadiSmart/cardiac/healthy_nii'

In [3]:
for pid in os.listdir(base_dir):
    pid_dir = os.path.join(base_dir, pid)
    for filename in os.listdir(pid_dir):
        dicom_path = os.path.join(pid_dir, filename)
        # print('dicom_path', dicom_path)
        
        try:
            # 如果是单个 DICOM 文件
            sitk_img = sitk.ReadImage(dicom_path)
            
            title = sitk_img.GetMetaData('0008|103e')
            type_name = sitk_img.GetMetaData('0054|0400')
        except:
            continue
        

        # if title == "CARDIAC SPECT" and "SA" in type_name:
        if not ("CardView Results" == title and "SA" in type_name):
            continue
        
        print(filename, title, type_name)
        
        out_dir = os.path.join(dst_dir, pid)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        
        img_arr = sitk.GetArrayFromImage(sitk_img)
        nb, height, width = img_arr.shape
        reshape_arr = img_arr.reshape(nb, height, -1, 20)
        print(reshape_arr.shape)
        
        # 将 NumPy 数组转换为 SimpleITK Image
        out_img = sitk.GetImageFromArray(reshape_arr, isVector=False)  # SimpleITK 处理 (T, Z, Y, X)

        # 设定体素间距，必须是 4 个值 (X, Y, Z, T)
        out_img.SetSpacing((1.0, 1.0, 1.0, 1.0))  # 时间间隔设置为 1.0

        # 保存为 NIfTI
        sitk.WriteImage(out_img, os.path.join(out_dir, f"{filename}_SA.nii.gz"))
        sitk.WriteImage(out_img, f"{filename}_From_SA.nii.gz")
        
        _, xs, ys, zs = reshape_arr.shape
        factor = 3
        scale_arr = np.zeros((nb, xs*factor, ys*factor, zs*factor))
        for i in range(nb):
            scale_arr[i] = scipy.ndimage.zoom(reshape_arr[i], factor, order=1) # order=1 表示双线性插值
        
        scale_arr
        # 将 NumPy 数组转换为 SimpleITK Image
        out_img2 = sitk.GetImageFromArray(scale_arr, isVector=False)  # SimpleITK 处理 (T, Z, Y, X)

        # 设定体素间距，必须是 4 个值 (X, Y, Z, T)
        out_img2.SetSpacing((1.0, 1.0, 1.0, 1.0))  # 时间间隔设置为 1.0

        # 保存为 NIfTI
        sitk.WriteImage(out_img2, os.path.join(out_dir, f"{filename}_From_SA_scaled.nii.gz"))
        
        
    # break






409 CardView Results STRESS_FBP(G)_SA
(9, 20, 19, 20)
161 CardView Results STRESS_FBP(G)_SA
(9, 20, 17, 20)
1088 CardView Results STRESS_FBP(G)_SA
(9, 20, 19, 20)
108 CardView Results STRESS_FBP(G)_SA
(9, 20, 19, 20)
1004 CardView Results STRESS_FBP(G)_SA
(9, 20, 21, 20)
